# Connect to AWS, Create EC2 Instance and Load Balancer 

## Note: 



# Import Python Libraries

In [15]:
import os
import boto3
from botocore.exceptions import ClientError 

# 1 & 2) Create Security Group & Authorize Traffic

In [16]:
group_name = "21484971-sg"
security_group_description="SHH traffic only - 21484971"
availability_zone = "ap-southeast-2"

## Rsource & Client Connection
ec2 = boto3.resource('ec2', availability_zone)
ec2_client = boto3.client("ec2", region_name=availability_zone)

# Try get the security group by name 
try:
    response = ec2_client.describe_security_groups(GroupNames=[group_name])
    for p in response['SecurityGroups']:
        print("Group Name: " + p['GroupName'])
        print("Group Description: " + p['Description'])
        print("GroupId: " + p['GroupId'])
        security_group_id = p['GroupId']
except ClientError as e: #Fails when the group does not exist 
    # Try to create security group with the provided id, catch error any errors
    try:
        # Create a security group and allow SSH inbound rule through the VPC
        securitygroup = ec2.create_security_group(GroupName=group_name, Description=security_group_description)
        securitygroup.authorize_ingress(CidrIp='0.0.0.0/0', IpProtocol='tcp', FromPort=22, ToPort=22)

        security_group_id = securitygroup.id
    
    except ClientError as e:
        print(e)

    print(security_group_id)


Group Name: 21484971-sg
Group Description: security group for development environment
GroupId: sg-091eb217391dc8774


# 3) Create Key Pair if not created if not, create one

In [ ]:
ec2_resources = boto3.resource('ec2')

key_file_name = "21484971-key.pem"
key_name = '21484971-key'

path = "./" + key_file_name

if( not os.path.isfile(path)):
    # create a file to store the key locally
    outfile = open(key_file_name,'w')

    # call the boto ec2 function to create a key pair
    key_pair = ec2_resources.create_key_pair(KeyName=key_name)

    # capture the key and store it in a file
    KeyPairOut = str(key_pair.key_material)
    print(KeyPairOut)

    outfile.write(KeyPairOut)

    ## Run Chmod 400 from python
    mode = os.stat(path).st_mode
    mode |= (mode & 400) >> 2    # copy R bits to X
    os.chmod(path, mode)


# 4) Create Connection, Instance & Tag

In [20]:
## Instance information
image_id = "ami-d38a4ab1"
instance_type = "t2.micro"
security_group = security_group_id #manually override if not creating a group before (sg-091eb217391dc8774)
# key_name = '21484971-key' #Can manually override if not running in steps

## Rsource connection
ec2 = boto3.resource('ec2',availability_zone)

## Create instance
instances = ec2.create_instances(
    ImageId=image_id, 
    InstanceType = instance_type,  
    SecurityGroupIds=[security_group],
    KeyName=key_name,
    MaxCount=1,
    MinCount=1
 )

## Store and return instance id
id = instances[0].instance_id
print(id)

## Tag instance 
ec2.create_tags(Resources=[id], Tags=[{'Key':'Name', 'Value':'21484971'}])

i-0841693e26cbf7187


[ec2.Tag(resource_id='i-0841693e26cbf7187', key='Name', value='21484971')]

# 5) Get Ip Address

In [11]:
ec2_client = boto3.client("ec2", region_name=availability_zone)
reservations = ec2_client.describe_instances(InstanceIds=[id]).get("Reservations")
for reservation in reservations:
    for instance in reservation['Instances']:
        ip_address=instance.get("PublicIpAddress")

print(ip_address)

3.104.106.39


# Use IP to Connect with SSH from terminal

In [11]:
## Bash Command to run
# ssh -i <KEY FILE NAME>.pem ubuntu@<IP ADDRESS>

print(path)
print(ip_address)


./21484971-key.pem
3.104.111.241


# Instance Functions from Python

## Stop instance 

In [41]:
response = ec2_client.stop_instances(InstanceIds=[id])
print(response)

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'}, 'InstanceId': 'i-06e77d0ea999001c2', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': 'c14aea5b-4481-46c8-9aad-eae09c37652c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'c14aea5b-4481-46c8-9aad-eae09c37652c', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '579', 'date': 'Sat, 14 Aug 2021 03:39:39 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


## Resart Instance

In [34]:
response = ec2_client.start_instances(InstanceIds=[id])
print(response)

{'StartingInstances': [{'CurrentState': {'Code': 16, 'Name': 'running'}, 'InstanceId': 'i-0a918e4da7277e787', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '60152f56-35de-4a31-831e-582667f42672', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '60152f56-35de-4a31-831e-582667f42672', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '580', 'date': 'Sat, 14 Aug 2021 03:35:12 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


## Terminate Instance

In [89]:
 response = ec2_client.terminate_instances(InstanceIds=[id])
 print(response)

{'TerminatingInstances': [{'CurrentState': {'Code': 32, 'Name': 'shutting-down'}, 'InstanceId': 'i-0e4248cee073abc9d', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': 'ce44334f-dac7-4fe8-bfc5-3d08e527ab3d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'ce44334f-dac7-4fe8-bfc5-3d08e527ab3d', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'transfer-encoding': 'chunked', 'vary': 'accept-encoding', 'date': 'Sat, 14 Aug 2021 09:33:56 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}
